In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate Cities List

In [19]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

613

## Perform API Calls

In [65]:
import openweathermapy.core as owm
from pprint import pprint
from urllib.error import HTTPError

# OpenWeatherMap API Key
api_key = api_keys.api_key

# Create settings dictionary with information we're interested in
settings = {"units": "imperial", "appid": api_key}

# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 

weather_data = []
city_idx = []

# Get current weather
for i, city in enumerate(cities):
    #print(f'{city}')
    try:
        current_weather = owm.get_current(city, **settings)
    except HTTPError:
        print(f"No weather found for city {i} {city}")
    else:
#         pprint(f"Current weather object for Paris: {current_weather}.")
#         print(f'temp--> {current_weather["main"]["temp"]}')
#         print(f'humidity --> {current_weather["main"]["humidity"]}')
#         print(f'clouds --> {current_weather["clouds"]["all"]}')
#         print(f'wind speed --> {current_weather["wind"]["speed"]}')
        print(f"Weather found for city {i} {city}")
        weather_data.append(current_weather)
        # pprint(weather_data)
    if (i >= 20):
        break

summary = ['id', 'name', "main.temp", "main.humidity", "clouds.all", 'wind.speed']
data = [response(*summary) for response in weather_data]
# print(len(weather_data))
# print(data)

weather_data_df = pd.DataFrame(data, columns=['City ID', 'City Name', 'Temp(F)', 'Humidity(%)', 'Clouds(%)', 'Wind Speed(mph)'])
weather_data_df

Weather found for city 0 hermanus
Weather found for city 1 norrtalje
Weather found for city 2 atuona
Weather found for city 3 clarence town
Weather found for city 4 port alfred
Weather found for city 5 sao filipe
Weather found for city 6 marsh harbour
Weather found for city 7 puro
Weather found for city 8 pevek
No weather found for city 9 maloshuyka
Weather found for city 10 hobart
Weather found for city 11 cidreira
Weather found for city 12 kapaa
Weather found for city 13 ler
Weather found for city 14 lagoa
No weather found for city 15 umzimvubu
Weather found for city 16 salalah
Weather found for city 17 qujing
No weather found for city 18 taolanaro
No weather found for city 19 tsihombe
Weather found for city 20 husavik


,City ID,City Name,Temp(F),Humidity(%),Clouds(%),Wind Speed(mph)
0,3366880,Hermanus,55.60,80,8,4.50
1,2688250,Norrtalje,78.80,69,0,10.29
2,4020109,Atuona,79.54,100,92,8.41
3,2171465,Clarence Town,62.60,45,0,6.93
4,964432,Port Alfred,63.70,83,8,18.25
5,3374210,Sao Filipe,76.48,96,0,7.40
6,3571913,Marsh Harbour,84.13,97,44,15.57
7,1706889,Puro,79.09,96,68,5.95
8,2122090,Pevek,56.14,65,0,14.12
9,2163355,Hobart,50.00,61,20,6.93


In [47]:
def google_api_request(url, params):
    params['key'] = gkey
    result = requests.get(url, params=params).json()
    if 'error_message' in result:
        raise Exception(result['error_message'])
    return result

def geocode_request(address):
    return google_api_request(
        "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=", {'address': address}
    )

def places_request(lat, lng, radius=8000, keyword=None, place_type=None, rankby=None):
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
    }
    if keyword:
        params['keyword'] = keyword
    if place_type:
        params['type'] = place_type
    if rankby:
        params['rankby'] = rankby
        del params['radius']
    return google_api_request(
        "https://maps.googleapis.com/maps/api/place/nearbysearch/json", params
    )